In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
with open('./arXiv_src_manifest.xml', 'r') as f:
    mani = ET.parse(f)

In [15]:
root = mani.getroot()
def parse_element(elem):
    return_dict = {}
    for e in elem:
        return_dict[e.tag] = e.text
    return return_dict
def parse_root(root):
        return [parse_element(child) for child in iter(root) if child.tag != 'timestamp']

filedf = pd.DataFrame(parse_root(root))
filedf[['num_items', 'size']] = filedf[['num_items', 'size']].astype(int)
filedf['filename'] = filedf['filename'].astype(str)

In [16]:
import hashlib 

In [71]:
hash_md5 = hashlib.md5()
with open('../arXiv_src_1804_001.tar', "rb") as f:
    for chunk in iter(lambda: f.read(4096), b""):
        hash_md5.update(chunk)
hash_md5.hexdigest()

'fe34df50d3426c745ea9a8b5ed924859'

In [72]:
filedf.loc[filedf['filename'].str.contains('1804_001')]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1742,d2ca6595ddb8055262ea1c6adf4cd4e3,src/arXiv_src_1804_001.tar,1804.00001,1804.00262,fe34df50d3426c745ea9a8b5ed924859,262,1,526207253,2018-06-05 04:56:42,1804


In [7]:
S = filedf['size']

In [9]:
S.sum()

924382365733

In [46]:
import subprocess
P = subprocess.run(['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006.tar', '..'], 
                   stderr=subprocess.PIPE,
                   stdout=subprocess.PIPE)

In [41]:
P.stdout

b"download: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\ndownload: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\n"

In [42]:
P.stderr

b'ERROR: S3 error: 404 (NoSuchKey): The specified key does not exist.\n'

In [45]:
P.check_returncode()

CalledProcessError: Command '['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006sab.tar', '..']' returned non-zero exit status 12.

In [19]:
df = filedf.head()
notdf = pd.DataFrame([{'filename': 'src/arXiv_src_0001_001.tar'}])

In [74]:
idx = df.filename.isin(notdf.filename)
df[~idx].iloc[0]

content_md5sum    d90df481661ccdd7e8be883796539743
filename                src/arXiv_src_0002_001.tar
first_item                         astro-ph0002001
last_item                          quant-ph0002094
md5sum            4592ab506cf775afecf4ad560d982a00
num_items                                     2365
seq_num                                          1
size                                     227036528
timestamp                      2010-12-23 00:18:09
yymm                                          0002
Name: 1, dtype: object

In [55]:
df.filename.isin(notdf.filename)

0    False
1     True
2    False
3    False
4    False
Name: filename, dtype: bool

In [57]:
df.to_csv('head_df.csv')

In [58]:
notdf.to_csv('notdf.csv')

In [35]:
nondf = pd.read_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv',index_col=0)
nondf.append(pd.DataFrame([{'filename':'hola'}]), ignore_index=True)

/usr/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Unnamed: 0.1,filename
0,NaN,hola


In [88]:
import os
os.path.ismount('/mnt')

True

In [39]:
notdf

,filename
0,src/arXiv_src_0001_001.tar


In [69]:
#filedf.to_csv('/home/luis/mountpoint/allfiles.csv')
# CAReFulll notdf.to_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv')

NameError: name 'notdf' is not defined

In [82]:
%load_ext autoreload
%autoreload 2

In [67]:
from dload import DownloadMan, parse_manifest
mountpoint = '/mnt/arXiv_src/'
allfiles = 'allfiles.csv'
doun = 'downloaded_log.csv'
error_log = 'error.log'
D = DownloadMan(mountpoint, allfiles, doun, error_log)

In [97]:
import process as P
xx = P.Xtraction('/mnt/arXiv_src/src/arXiv_src_1804_001.tar')

untaring file: /mnt/arXiv_src/src/arXiv_src_1804_001.tar
querying the arxiv
query successful


In [102]:
xx.tar_path

'/mnt/arXiv_src/src/arXiv_src_1804_001.tar'

In [105]:
xx.path_dir('../mathy')

'../mathy/1804_001'

In [99]:
D.check_md5()

NameError: name 'D' is not defined

In [68]:
new_all = parse_manifest('/mnt/arXiv_src/src/arXiv_src_manifest.xml')

In [12]:
fname = 'src/arXiv_src_1607_002.tar'
D.md5sum(fname)

'1b621f41cd46b9d3388351ba9e3aefaf'

In [13]:
D.allfiles_df[D.allfiles_df.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,8979b2424c6d31c8be791aa675ef05ba,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,a29303e80c4f753e17209a9f6238f2c5,526,2,545216578,2017-07-05 05:41:14,1607


In [14]:
new_all[new_all.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,2446538d56ddee4f0e28ee65937b6189,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,1b621f41cd46b9d3388351ba9e3aefaf,526,2,556653925,2018-07-05 06:37:36,1607


In [22]:
D.allfiles_df[D.allfiles_df.filename == fname].md5sum.iloc[0]

'5c0c499b3cc7c4961e522aa55e8974a5'

In [24]:
sum([D.filesize(fn[0]) for fn in list(D.downloaded_df.values)])/1e9

116.614211313

In [ ]:
keep_going = 0
while keep_going == 0:
    keep_going = D.get_next()
    if D.check_md5():
        pass
    else:
        break

Dowloand of file src/arXiv_src_0003_001.tar successful
Dowloand of file src/arXiv_src_0004_001.tar successful
Dowloand of file src/arXiv_src_0005_001.tar successful
Dowloand of file src/arXiv_src_0006_001.tar successful
Dowloand of file src/arXiv_src_0007_001.tar successful
Dowloand of file src/arXiv_src_0008_001.tar successful
Dowloand of file src/arXiv_src_0009_001.tar successful
Dowloand of file src/arXiv_src_0010_001.tar successful
Dowloand of file src/arXiv_src_0011_001.tar successful
Dowloand of file src/arXiv_src_0012_001.tar successful
Dowloand of file src/arXiv_src_0101_001.tar successful
Dowloand of file src/arXiv_src_0102_001.tar successful
Dowloand of file src/arXiv_src_0103_001.tar successful
Dowloand of file src/arXiv_src_0104_001.tar successful
Dowloand of file src/arXiv_src_0105_001.tar successful
Dowloand of file src/arXiv_src_0106_001.tar successful
Dowloand of file src/arXiv_src_0107_001.tar successful
Dowloand of file src/arXiv_src_0108_001.tar successful
Dowloand o

In [45]:
import urllib.request as req
url = 'http://export.arxiv.org/api/query?search_query=id:1804.00020&start=0&max_results=5'
data = req.urlopen(url).read()
root = ET.fromstring(data)
res = root.getchildren()[-1]
res.find('title').text

AttributeError: 'NoneType' object has no attribute 'text'

In [38]:
for el in res.getchildren():
    print(el.tag)

{http://www.w3.org/2005/Atom}id
{http://www.w3.org/2005/Atom}updated
{http://www.w3.org/2005/Atom}published
{http://www.w3.org/2005/Atom}title
{http://www.w3.org/2005/Atom}summary
{http://www.w3.org/2005/Atom}author
{http://www.w3.org/2005/Atom}author
{http://arxiv.org/schemas/atom}comment
{http://www.w3.org/2005/Atom}link
{http://www.w3.org/2005/Atom}link
{http://arxiv.org/schemas/atom}primary_category
{http://www.w3.org/2005/Atom}category


In [24]:
import sys
sys.path.insert(0,'arxiv.py/')
import arxiv

In [218]:
import arxiv
D = arxiv.query(id_list=['1805.02612'])
for d in D[0].keys():
    print(d,D[0][d])

id http://arxiv.org/abs/1805.02612v2
guidislink True
updated 2018-06-27T06:49:34Z
updated_parsed time.struct_time(tm_year=2018, tm_mon=6, tm_mday=27, tm_hour=6, tm_min=49, tm_sec=34, tm_wday=2, tm_yday=178, tm_isdst=0)
published 2018-05-07T16:45:08Z
published_parsed time.struct_time(tm_year=2018, tm_mon=5, tm_mday=7, tm_hour=16, tm_min=45, tm_sec=8, tm_wday=0, tm_yday=127, tm_isdst=0)
title Infinitely many new families of complete cohomogeneity one
  G_2-manifolds: G_2 analogues of the Taub-NUT and Eguchi-Hanson spaces
title_detail {'type': 'text/plain', 'language': None, 'base': 'http://export.arxiv.org/api/query?search_query=&id_list=1805.02612&start=0&max_results=10&sortBy=relevance&sortOrder=descending', 'value': 'Infinitely many new families of complete cohomogeneity one\n  G_2-manifolds: G_2 analogues of the Taub-NUT and Eguchi-Hanson spaces'}
summary We construct infinitely many new 1-parameter families of simply connected
complete noncompact G_2-manifolds with controlled geomet

In [219]:
D[0].links

[{'href': 'http://arxiv.org/abs/1805.02612v2',
  'rel': 'alternate',
  'type': 'text/html'},
 {'title': 'pdf',
  'href': 'http://arxiv.org/pdf/1805.02612v2',
  'rel': 'related',
  'type': 'application/pdf'}]

In [221]:
arxiv.query(id_list=['1805.02773'])[0].links

[{'href': 'http://arxiv.org/abs/1805.02773v1',
  'rel': 'alternate',
  'type': 'text/html'},
 {'title': 'pdf',
  'href': 'http://arxiv.org/pdf/1805.02773v1',
  'rel': 'related',
  'type': 'application/pdf'}]

In [26]:
year_month = '1804.'
id_list = [year_month + str(k).zfill(5) for k in range(1,263)]
dicts = arxiv.query(id_list=id_list, max_results=300)

In [89]:
diff_geom = [d for d in dicts if d.get('tags',None)[0]['term']=='math.DG']
diff_geom

[{'id': 'http://arxiv.org/abs/1804.00018v1',
  'guidislink': True,
  'updated': '2018-03-30T18:11:49Z',
  'updated_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=18, tm_min=11, tm_sec=49, tm_wday=4, tm_yday=89, tm_isdst=0),
  'published': '2018-03-30T18:11:49Z',
  'published_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=18, tm_min=11, tm_sec=49, tm_wday=4, tm_yday=89, tm_isdst=0),
  'title': 'Uniqueness of convex ancient solutions to mean curvature flow in higher\n  dimensions',
  'title_detail': {'type': 'text/plain',
   'language': None,
   'base': 'http://export.arxiv.org/api/query?search_query=&id_list=1804.00001%2C1804.00002%2C1804.00003%2C1804.00004%2C1804.00005%2C1804.00006%2C1804.00007%2C1804.00008%2C1804.00009%2C1804.00010%2C1804.00011%2C1804.00012%2C1804.00013%2C1804.00014%2C1804.00015%2C1804.00016%2C1804.00017%2C1804.00018%2C1804.00019%2C1804.00020%2C1804.00021%2C1804.00022%2C1804.00023%2C1804.00024%2C1804.00025%2C1804.00026%2C1

In [90]:
[d.title for d in diff_geom]

['Uniqueness of convex ancient solutions to mean curvature flow in higher\n  dimensions',
 'An almost splitting theorem for a warped product space',
 'Surfaces of revolution with prescribed mean and skew curvatures in\n  Lorentz-Minkowski space']

In [29]:
math = [d for d in dicts if 'math' in d['tags'][0]['term']]

In [43]:
set([d['tags'][0]['term'] for d in math])

{'math-ph',
 'math.AG',
 'math.AP',
 'math.CO',
 'math.CV',
 'math.DG',
 'math.DS',
 'math.FA',
 'math.HO',
 'math.KT',
 'math.LO',
 'math.MG',
 'math.NA',
 'math.NT',
 'math.OA',
 'math.OC',
 'math.PR',
 'math.QA',
 'math.RA',
 'math.RT',
 'math.SP',
 'math.ST'}

In [86]:
import re
def api2tar(id_str):
    '''
    The arxiv API produces ids in the format:
    http://arxiv.org/abs/1804.00018v1',
 http://arxiv.org/abs/1804.00193v1',
 http://arxiv.org/abs/1804.00259v2
    The tar files have name format:
    <TarInfo '1804/1804.00020.gz' at 0x7f0ba68c8cc8>
    '''
    name = re.match(r'.*([0-9]{4})\.([0-9]{5})',id_str)
    return name.group(1) + '/' + name.group(1) + '.' + name.group(2) + '.gz'
    
    
[api2tar(d.id) for d in dicts if d.get('tags',None)[0]['term']=='math.DG']

['1804/1804.00018.gz', '1804/1804.00193.gz', '1804/1804.00259.gz']

In [78]:
DD = dicts[0]
DD

{'id': 'http://arxiv.org/abs/1804.00001v1',
 'guidislink': True,
 'updated': '2018-03-30T11:08:38Z',
 'updated_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=11, tm_min=8, tm_sec=38, tm_wday=4, tm_yday=89, tm_isdst=0),
 'published': '2018-03-30T11:08:38Z',
 'published_parsed': time.struct_time(tm_year=2018, tm_mon=3, tm_mday=30, tm_hour=11, tm_min=8, tm_sec=38, tm_wday=4, tm_yday=89, tm_isdst=0),
 'title': 'Phenomenological analysis of densification mechanism during spark plasma\n  sintering of MgAl2O4',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'http://export.arxiv.org/api/query?search_query=&id_list=1804.00001%2C1804.00002%2C1804.00003%2C1804.00004%2C1804.00005%2C1804.00006%2C1804.00007%2C1804.00008%2C1804.00009%2C1804.00010%2C1804.00011%2C1804.00012%2C1804.00013%2C1804.00014%2C1804.00015%2C1804.00016%2C1804.00017%2C1804.00018%2C1804.00019%2C1804.00020%2C1804.00021%2C1804.00022%2C1804.00023%2C1804.00024%2C1804.00025%2C1804.00026%2C18

In [6]:
import tarfile

with tarfile.open('/home/luis/MisDocumentos/arxivBulkDownload/arXiv_src_1804_001.tar') as ff:
    art_lst = [k for k in ff.getmembers()]

In [80]:
ti = art_lst[1]
ti

<TarInfo '1804/1804.00020.gz' at 0x7f0ba68c8cc8>

In [52]:
ti.get_info()

{'name': '1804/1804.00020.gz',
 'mode': 420,
 'uid': 10519,
 'gid': 10519,
 'size': 15177,
 'mtime': 1522713637,
 'chksum': 5742,
 'type': b'0',
 'linkname': '',
 'uname': 'e-prints',
 'gname': 'e-prints',
 'devmajor': 0,
 'devminor': 0}

In [45]:
import gzip
with tarfile.open('/home/luis/MisDocumentos/arxivBulkDownload/arXiv_src_1804_001.tar') as ff:
    zhu_gz = ff.extractfile('1804/1804.00020.gz')
    zhu = gzip.open(zhu_gz,'rb')
    zhu_str = zhu.read()

In [95]:
with open('./zhu_untared.tex', 'w') as f:
    f.write(zhu_str.decode('utf-8'))

In [93]:
print(zhu_str)

b"\\documentclass[psamsfonts,a4paper]{amsart}\r\n\r\n%-------Packages---------\r\n\\usepackage{amssymb,amsfonts}\r\n\\usepackage{amsaddr}\r\n\\usepackage[all,arc]{xy}\r\n\\usepackage{enumerate}\r\n%\\usepackage{mathrsfs}\r\n\\usepackage{tikz}\r\n\\usepackage{parskip}\r\n\\usepackage[margin=2.5cm]{geometry}\r\n\\usepackage{eucal}\r\n\r\n%--------Theorem Environments--------\r\n%theoremstyle{plain} --- default\r\n\r\n\r\n\\newtheorem{thm}{Theorem}[section]\r\n\r\n% Reinsert ``[section]'' here if you want theorems numbered by section.\r\n\r\n\\newtheorem{cor}[thm]{Corollary}\r\n\\newtheorem{prop}[thm]{Proposition}\r\n\\newtheorem{lemma}[thm]{Lemma}\r\n\\newtheorem{conj}[thm]{Conjecture}\r\n\\newtheorem{quest}[thm]{Question}\r\n\r\n\\theoremstyle{definition}\r\n\\newtheorem{defn}[thm]{Definition}\r\n\\newtheorem{defns}[thm]{Definitions}\r\n\\newtheorem{con}[thm]{Construction}\r\n\\newtheorem{exmp}[thm]{Example}\r\n\\newtheorem{exmps}[thm]{Examples}\r\n\\newtheorem{notn}[thm]{Notation}\r\n\

In [243]:
import gzip
import chardet
file_tar = tarfile.open('/mnt/arXiv_src/src/arXiv_src_1708_017.tar')
tar2 = file_tar.extractfile('1708/1708.06614.gz')
#print(chardet.detect(tar2))
#with open('../todo_macizo.pdf','w') as todo_macizo:
#    todo_macizo.write(tar2.read().decode('utf-8'))
with tarfile.open(fileobj=tar2) as tars:
    for t in tars:
        print(t.get_info()['name'])
#gz = gzip.open(tar2, 'rb')
#with gzip.open(tar2) as tars:
#    t = tars.read()
#print(t.decode('utf-8'))
#chardet.detect(t)

#with open('../holita.txt','w') as holaf:
#    holaf.write(allfile.read().decode('utf-8'))
    

ReadError: file could not be opened successfully

In [245]:
tarfile.ReadError()

tarfile.ReadError()

In [238]:
with tarfile.open('/mnt/arXiv_src/src/arXiv_src_1708_017.tar') as morgan:
    for mor in morgan:
        print(mor.name)


1708
1708/1708.06770.gz
1708/1708.06466.gz
1708/1708.06631.gz
1708/1708.06467.gz
1708/1708.06796.gz
1708/1708.06536.gz
1708/1708.06588.gz
1708/1708.06447.gz
1708/1708.06760.gz
1708/1708.06797.gz
1708/1708.06553.gz
1708/1708.06564.gz
1708/1708.06657.gz
1708/1708.06502.gz
1708/1708.06566.gz
1708/1708.06526.gz
1708/1708.06479.gz
1708/1708.06570.gz
1708/1708.06713.gz
1708/1708.06780.gz
1708/1708.06601.pdf
1708/1708.06711.gz
1708/1708.06599.gz
1708/1708.06712.gz
1708/1708.06605.gz
1708/1708.06636.pdf
1708/1708.06789.gz
1708/1708.06758.gz
1708/1708.06775.gz
1708/1708.06730.gz
1708/1708.06620.gz
1708/1708.06518.pdf
1708/1708.06670.gz
1708/1708.06700.gz
1708/1708.06475.gz
1708/1708.06490.gz
1708/1708.06523.gz
1708/1708.06672.gz
1708/1708.06668.gz
1708/1708.06471.gz
1708/1708.06571.gz
1708/1708.06759.pdf
1708/1708.06737.pdf
1708/1708.06698.gz
1708/1708.06489.gz
1708/1708.06783.gz
1708/1708.06692.gz
1708/1708.06598.gz
1708/1708.06615.gz
1708/1708.06485.gz
1708/1708.06632.gz
1708/1708.06575.gz
17

In [208]:
file_tar.extract?

In [109]:
tarfile.TarFile.extractall?

In [157]:
import chardet
chardet.detect(t)

{'encoding': 'Windows-1252', 'confidence': 0.7292205811322673, 'language': ''}

In [222]:
import glob
glob.glob('/mnt/arXiv_src/src/arXiv_src_1805_*')

['/mnt/arXiv_src/src/arXiv_src_1805_001.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_003.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_004.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_005.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_006.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_007.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_008.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_009.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_010.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_011.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_012.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_013.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_014.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_015.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_016.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_017.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_019.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_020.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_021.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_022.tar',
 '/mnt/arXiv_src/src/arXiv_src_1805_023.tar',
 '/mnt/arXiv_src/src/arXiv_src_180

In [172]:
bytes.decode?

In [171]:
type(t)

bytes